# Seminar Python 4: Pachetul scikit-learn

Exemplu instalare pachet scikit-learn: în (Windows) Command prompt / (UNIX) fereastră Terminal / shell se rulează:
`pip install scikit-learn`

Similar și pentru alte pachete Python: scipy, six, cycler, pyparsing, kiwisolver, python-dateutil, matplotlib, pytz, pandas, seaborn, numpy, sklearn, statsmodels etc.  
După caz, poate fi necesar upgrade-ul PIP (Python package installer) - v. mai multe detalii pe https://datatofish.com/upgrade-pip/

In [ ]:
#Verificare pachet/modul instalat (tratare excepție în caz de eșec import cu try...except)
try:                 
    import sklearn
    print('Import OK.')
except ImportError as err: 
    print(err)

In [ ]:
#instalare in Jupyter Lab: #"comandă magică" IPython, rulează în shell
%pip install scikit-learn

## Clusterizare K-Means în Python cu pachetul scikit-learn
Metoda de clusterizare K-means realizează gruparea observațiilor dintr-un set de date într-un număr de K "clustere" - grupuri de observații care prezintă similarități.

In [ ]:
#Exemplu 1. Gruparea unui set de valori în 3 clustere
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

X = np.array([[5,3],
     [10,15],
     [15,12],
     [24,10],
     [30,45],
     [85,70],
     [71,80],
     [60,78],
     [55,52],
     [80,91]])
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
print(kmeans.cluster_centers_)
print(kmeans.labels_)
f1 = plt.figure()
plt.scatter(X[:,0],X[:,1], label='True Position')
f2 = plt.figure()
plt.scatter(X[:,0], X[:,1], c=kmeans.labels_, cmap='rainbow')
f3 = plt.figure()
plt.scatter(kmeans.cluster_centers_[:,0] ,kmeans.cluster_centers_[:,1], color='black')
plt.show()

Această metodă este utilizată în învățarea automată (_machine learning_) pentru a identifica clusterele pe baza unui set de antrenare (train); pe această bază se vor face predicții privind apartenența altor observații (din setul de test) la clusterele identificate (v. mai multe detalii [aici](https://towardsdatascience.com/understanding-k-means-clustering-in-machine-learning-6a6e67336aa1))

**Aplicație**  
Scufundarea Titanicului în 1912 a dus la 1502 victime din cei 2224 pasageri și membri ai echipajului.
Se vor utiliza două seturi de date `train.csv` și `test.csv`, ce conțin informații legate de pasageri. Diferența majoră dintre cele două seturi la nivel de coloane constă în coloana `Survived`,  prezentă doar în setul de antrenare. Se poate considera că supraviețuirea a fost influențată de anumite atribute, cum ar fi vârsta, sexul, clasa biletului de călătorie etc. Pornind de la aceste caracteristici, se vor grupa (clusteriza) pasagerii din setul de date de antrenare în două clustere (supraviețuitori / nesupraviețuitori); pe baza acestora se vor face predicții privind apartenența pasagerilor din setul de test la unul dintre clustere.

In [ ]:
#Exemplu 2. Pas 1. Import biblioteci
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Exemplu 2. Pas 2. Citirea fișierelor și afișarea primelor 5 înregistrări
pd.options.display.max_columns = 12
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

print('*****test*****')
print(test.head())
print('*****train*****')
print(train.head())

In [ ]:
#Exemplu 2. Pas 3. Calcul statistici de bază
print('*****test_stats*****')
print(test.describe())
print('*****train_stats*****')
print(train.describe())

Anumiți algoritmi machine learning, inclusiv k-means, nu permit valori lipsă. Astfel, vor fi identificate valorile lipsă.

In [ ]:
#Exemplu 2. Pas 4. Vizualizare denumire coloane din setul train și indentificare valori lipsă
print(train.columns.values)

print('*****train_ valori_lipsă *****')
print(train.isna())
print('*****test_valori_lipsă*****')
print(test.isna())

In [ ]:
#Exemplu 2. Pas 5. Calcul număr valori lipsă pentru coloanele celor două seturi de date
print("*****In the train set*****")
print(train.isna().sum())
print("\n")
print('*****In the test set*****')
print(test.isna().sum())

In [ ]:
#Exemplu 2. Pas 6. Înlocuirea valorilor lipsă cu media coloanei, utilizând fillna()
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)
print(train.isna().sum())
print(test.isna().sum())

In [ ]:
#Exemplu 2. Pas 7. Evaluarea supraviețuitorilor în funcție de Pclass, Sex, SibSp
print(train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print(train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False))

In [ ]:
#Exemplu 2. Pas 8. Reprezentare grafică a coloanelor Age-Survived și Pclass-Survived
g = sns.FacetGrid(train, col='Survived')
g.map(plt.hist, 'Age')
grid = sns.FacetGrid(train, col='Survived', row='Pclass')
grid.map(plt.hist, 'Age')
grid.add_legend()
plt.show()

In [ ]:
#Exemplu 2. Pas 9. Afișarea informațiilor legate de setul de date train
train.info()

In [ ]:
#Exemplu 2. Pas 10. Eliminarea coloanelor non-numerice care nu influențează supraviețuirea
train = train.drop(['Name','Ticket', 'Cabin','Embarked'], axis=1)
test = test.drop(['Name','Ticket', 'Cabin','Embarked'], axis=1)

In [ ]:
#Exemplu 2. Pas 11. Transformarea tipului de dată a coloanei Sex
labelEncoder = LabelEncoder()
labelEncoder.fit(train['Sex'])
labelEncoder.fit(test['Sex'])
train['Sex'] = labelEncoder.transform(train['Sex'])
test['Sex'] = labelEncoder.transform(test['Sex'])
train.info()
test.info()

In [ ]:
#Exemplu 2. Pas 12. Var. X este un vector (array din pachetul numpy) identic cu setul train,
# din care a fost ștearsă coloana Survived, iar y este un vector format din coloana Survived
X = np.array(train.drop(['Survived'], 1).astype(float))
y = np.array(train['Survived'])

In [ ]:
#Exemplu 2. Pas 13. Aplearea metodei Kmeans și setarea parametrului n_clusters = 2
# (supraviețuitori/nesupraviețuitori). 
kmeans = KMeans(n_clusters=2) 
#kmeans = KMeans(n_clusters=2, max_iter=600)
kmeans.fit(X)

#Alte setări (argumente) posibile pentru kmeans:
'''KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)'''

In [ ]:
#Exemplu 2. Pas 14. Evaluarea rezultatelor
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

Modelul clusterizează correct cu o acuratețe de circa 50%.  
Pentru o îmbunățire a rezultatelor, se vor scala datele de intrare.  
Exemplul 2, pas 15: deschideți și rulați `Ex_2_15.ipynb`

In [ ]:
# --- NU EXECUTAȚI ACEASTĂ CELULĂ --- #
#Exemplu 2. Cod complet exemplu KMmeans - salvat și ca script (fișier) ex_2.py,
#pentru rulare separată (în fereastră Terminal / IDLE / PyCharm)

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 12
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

print('*****test*****')
print(test.head())
print('*****train*****')
print(train.head())

print('*****test_stats*****')
print(test.describe())
print('*****train_stats*****')
print(train.describe())

print(train.columns.values)

print(train.isna())
print(test.isna())

print('*****In the train set*****')
print(train.isna().sum())
print("\n")
print('*****In the test set*****')
print(test.isna().sum())

train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)
print(train.isna().sum())
print(test.isna().sum())


print(train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print(train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print(train[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False))

g = sns.FacetGrid(train, col='Survived')
g.map(plt.hist, 'Age')
grid = sns.FacetGrid(train, col='Survived', row='Pclass')
grid.map(plt.hist, 'Age')
grid.add_legend()
plt.show()

train.info()

train = train.drop(['Name','Ticket', 'Cabin','Embarked'], axis=1)
test = test.drop(['Name','Ticket', 'Cabin','Embarked'], axis=1)

labelEncoder = LabelEncoder()
labelEncoder.fit(train['Sex'])
labelEncoder.fit(test['Sex'])
train['Sex'] = labelEncoder.transform(train['Sex'])
test['Sex'] = labelEncoder.transform(test['Sex'])

train.info()

test.info()

X = np.array(train.drop(['Survived'], 1).astype(float))

y = np.array(train['Survived'])

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=2)
kmeans.fit(X_scaled)
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=600,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

## Regresie logistică
Această metodă de machine learning este utilizată pentru probleme de clasificare - predicția apartenenței sau nu a unei observații la o clasă (regresie logistică binară) - de ex. pasager supraviețuitor sau nu.  
Se urmărește determinarea unei ecuații de regresie bazată pe funcția sigmoidă (funcția logistică), în loc de o dreaptă ca la regresia liniară (v. mai multe detalii [aici](https://towardsdatascience.com/introduction-to-logistic-regression-66248243c148)).  
Ecuația de regresie este estimată pe baza datelor din setul de antrenare (train). Apoi se vor face predicții asupra setului de test.

In [ ]:
#Exemplu 3. Regresie logistică (logistic regression)
import pandas as pd
from sklearn.linear_model import LogisticRegression
pd.options.display.max_columns = 12
test = pd.read_csv('test1.csv')
train = pd.read_csv('train.csv')

print('*****test*****')
print(test[:4])
print('*****train*****')
print(train[:4])

train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)
print(train.isna().sum())
print(test.isna().sum())

train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
y_test= test['Survived'].values
print(X_train[:5])
print(y_train[:5])

model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
verbose=0, warm_start=False)

model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(y_predict)

print((y_test == y_predict).mean())

## Regresie liniară simplă

In [ ]:
#Exemplu 4. Regresie simplă liniară (Ordinary Least Squares)
import pandas as pd
import statsmodels.api as sm
pd.options.display.max_columns = 12
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

X_train = train['IsFemale'].values
X_train = sm.add_constant(X_train)

y_train = train['Survived'].values

model = sm.OLS(y_train, X_train)

results = model.fit()
print(results.params)
print(results.summary())

## Regresie liniară multiplă

In [ ]:
#Exemplu 5. Regresie multiplă (multiple regression)
import pandas as pd
import statsmodels.formula.api as smf

pd.options.display.max_columns = 12
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')


train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)


X = pd.DataFrame(train, columns=['Pclass', 'IsFemale', 'Age'])
y = train['Survived']

results = smf.ols('y ~ Pclass + IsFemale + Age', data=train).fit()
print(results.params)

print(round(results.predict(train[:5])))

## Referințe
1. J. VanderPlas, Python Data Science Handbook: https://jakevdp.github.io/PythonDataScienceHandbook/index.html, Cap. 5
1. https://stackabuse.com/k-means-clustering-with-scikit-learn/ 
2. https://www.datacamp.com/community/tutorials/k-means-clustering-python
3. Wes McKinney, 2nd Edition of Python for Data Analysis DATA WRANGLING WITH PANDAS, NUMPY, AND IPYTHON, O’Reilley
4. https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303bc 
5. https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html 